In [ ]:
pip install tableauserverclient

In [ ]:
import tableauserverclient as TSC
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def main():
    # Step 1: Load Tableau Server details from environment variables
    tableau_server = os.getenv("TABLEAU_SERVER")
    tableau_username = os.getenv("TABLEAU_USERNAME")
    tableau_password = os.getenv("TABLEAU_PASSWORD")
    tableau_site = os.getenv("TABLEAU_SITE")
    view_name = input("Enter the name of the Tableau view to search for: ").strip()

    # Step 2: Tableau Server Authentication
    try:
        tableau_auth = TSC.TableauAuth(tableau_username, tableau_password, None)
        server = TSC.Server(tableau_server)
        server.use_server_version()
        print("Authenticating with Tableau Server...")

        with server.auth.sign_in(tableau_auth):
            print("Authentication successful!")

            # Step 3: Search for the view by name
            print(f"Searching for views with name containing '{view_name}'...")

            # Fetch all views
            all_views, pagination_item = server.views.get()
            matching_views = [
                view for view in all_views if view_name.lower() in view.name.lower()
            ]

            if matching_views:
                # Fetch all projects to match project names
                all_projects, _ = server.projects.get()

                print(f"Found {len(matching_views)} matching view(s):\n")
                for idx, view in enumerate(matching_views, start=1):
                    # Fetch related details about the workbook
                    workbook = server.workbooks.get_by_id(view.workbook_id)

                    # Match project name using the workbook's project ID
                    project_name = next(
                        (project.name for project in all_projects if project.id == workbook.project_id),
                        "Unknown Project"
                    )

                    # Construct the View URL
                    view_url = f"{tableau_server}/#/views/{view.content_url}"

                    print(f"{idx})")
                    print(f"- View Name: {view.name}")
                    print(f"- View ID: {view.id}")
                    print(f"- Workbook Name: {workbook.name}")
                    print(f"- Project Name: {project_name}")
                    print(f"- View URL: {view_url}")
                    print()
            else:
                print(f"No views found matching '{view_name}'.")

    except TSC.ServerResponseError as e:
        print(f"Server response error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    main()

Authenticating with Tableau Server...
Authentication successful!
Searching for views with name containing 'DASHBOARD'...
Found 6 matching view(s):

1)
- View Name: Dashboard
- View ID: a300e744-f45c-460c-bcde-d22fb0b7e98a
- Workbook Name: 지역별 매장 리포트
- Project Name: Sample Dashboard (v2.0)
- View URL: https://tableau-report.com/#/views/Outback_CRMReport_Draft_20180831_Upload/sheets/Dashboard

2)
- View Name: Seasonal Dashboard(2018.09)
- View ID: 07a3cf31-a4ef-4b92-a955-244038a247ac
- Workbook Name: Seasonal Dashboard
- Project Name: Sample Dashboard (v2.0)
- View URL: https://tableau-report.com/#/views/SeasonalDashboardadidas/sheets/SeasonalDashboard2018_09

3)
- View Name: Dashboard
- View ID: 558bbc28-8661-47b5-ad43-b91736f2a6a1
- Workbook Name: Monthly dashboard (Reebok)
- Project Name: One page dashboard
- View URL: https://tableau-report.com/#/views/MonthlydashboardReebok/sheets/Dashboard

4)
- View Name: Dashboard(Mobile)
- View ID: 36ed2057-aa83-4b59-8e32-28a148519b5c
- Workbook